# Simple Scheduling

In [1]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds


num_agents = 3
num_rounds = 3
num_slots = 3

num_vars = num_agents * num_rounds * num_slots
print(f"number of variables: {num_vars}")
# 27

number of variables: 27


In [2]:
def var_index(agent: int, round: int, slot: int) -> int:
    """Convert (agent, round, slot) to an array index"""
    return agent * num_rounds * num_slots + round * num_slots + slot


# Variable index for agent 1, round 2, slot 0
example_idx = var_index(1, 2, 0)
print(f"variable index = {example_idx}")

variable index = 15


In [3]:
c = np.zeros(num_vars)

In [4]:
integrality = np.ones(num_vars)
print(integrality)  # 1 means integer variable for `milp`

bounds = Bounds(lb=np.zeros(num_vars), ub=np.ones(num_vars))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]


In [5]:
A_eq = []
b_eq = []

for round in range(num_rounds):
    for slot in range(num_slots):
        constraint = np.zeros(num_vars)

        for agent in range(num_agents):
            constraint[var_index(agent, round, slot)] = 1

        A_eq.append(constraint)
        b_eq.append(1)  # Sum must equal 1

In [6]:
A_ub = []
b_ub = []

for agent in range(num_agents):
    for round in range(num_rounds):
        constraint = np.zeros(num_vars)

        for slot in range(num_slots):
            constraint[var_index(agent, round, slot)] = 1

        A_ub.append(constraint)
        b_ub.append(1)

In [7]:
total_slots = num_slots * num_rounds
min_slots_per_agent = total_slots // num_agents
max_slots_per_agent = min_slots_per_agent + (1 if total_slots % num_agents != 0 else 0)


A_range = []
b_lb = []
b_ub_range = []


for agent in range(num_agents):
    constraint = np.zeros(num_vars)
    for round in range(num_rounds):
        for slot in range(num_slots):
            constraint[var_index(agent, round, slot)] = 1

    A_range.append(constraint)
    b_lb.append(min_slots_per_agent)
    b_ub_range.append(max_slots_per_agent)

In [8]:
A_eq = np.array(A_eq)
b_eq = np.array(b_eq)
A_ub = np.array(A_ub)
b_ub = np.array(b_ub)
A_range = np.array(A_range)
b_lb = np.array(b_lb)
b_ub_range = np.array(b_ub_range)


constraints = [
    LinearConstraint(A_eq, b_eq, b_eq),
    LinearConstraint(A_ub, -np.inf, b_ub),
    LinearConstraint(A_range, b_lb, b_ub_range),
]


In [9]:
result = milp(c=c, integrality=integrality, bounds=bounds, constraints=constraints)
print(result.message)

Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [10]:
solution = result.x
for round in range(num_rounds):
    print(f"\nRound {round}:")
    for agent in range(num_agents):
        for slot in range(num_slots):
            if solution[var_index(agent, round, slot)] > 0:
                print(f"  Agent {agent} -> Slot {slot}")

# Round 0:
#   Agent 0 -> Slot 1
#   Agent 1 -> Slot 2
#   Agent 2 -> Slot 0

# Round 1:
#   Agent 0 -> Slot 0
#   Agent 1 -> Slot 2
#   Agent 2 -> Slot 1

# Round 2:
#   Agent 0 -> Slot 1
#   Agent 1 -> Slot 2
#   Agent 2 -> Slot 0


Round 0:
  Agent 0 -> Slot 1
  Agent 1 -> Slot 2
  Agent 2 -> Slot 0

Round 1:
  Agent 0 -> Slot 0
  Agent 1 -> Slot 2
  Agent 2 -> Slot 1

Round 2:
  Agent 0 -> Slot 1
  Agent 1 -> Slot 2
  Agent 2 -> Slot 0


# Job Shop Scheduling

In [11]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

# Customer appointment data
# Each customer has a list of (agent, duration) tuples representing appointments
customers_data = [
    [(0, 3), (1, 2), (2, 2)],  # Customer 0: needs 3 appointments
    [(0, 2), (2, 1), (1, 4)],  # Customer 1: needs 3 appointments
    [(1, 4), (2, 3)],  # Customer 2: needs 2 appointments
]

# Calculate problem dimensions
num_customers = len(customers_data)
num_appointments_total = sum(len(customer) for customer in customers_data)
num_agents = 1 + max(appt[0] for customer in customers_data for appt in customer)

# Calculate horizon (upper bound for appointment start times)
horizon = sum(appt[1] for customer in customers_data for appt in customer)

In [12]:
# Create appointment indexing
appointment_info = []  # List of (customer_id, appt_id, agent, duration)
appt_to_idx = {}  # Map (customer_id, appt_id) to variable index

idx = 0
for customer_id, customer in enumerate(customers_data):
    for appt_id, (agent, duration) in enumerate(customer):
        appointment_info.append((customer_id, appt_id, agent, duration))
        appt_to_idx[(customer_id, appt_id)] = idx
        idx += 1

# Find pairs of appointments for the same agent (for no-overlap constraints)
agent_appt_pairs = []
for i in range(num_appointments_total):
    for j in range(i + 1, num_appointments_total):
        if appointment_info[i][2] == appointment_info[j][2]:  # Same agent
            agent_appt_pairs.append((i, j))

num_binary_vars = len(agent_appt_pairs)

# Total variables: start times + makespan + binary ordering variables
num_vars = num_appointments_total + 1 + num_binary_vars


# Variable indexing functions
def start_var_idx(customer_id, appt_id):
    """Get index for appointment start time variable"""
    return appt_to_idx[(customer_id, appt_id)]


def makespan_idx():
    """Get index for makespan variable"""
    return num_appointments_total


def binary_idx(pair_idx):
    """Get index for binary ordering variable"""
    return num_appointments_total + 1 + pair_idx

In [13]:
c = np.zeros(num_vars)
c[makespan_idx()] = 1

In [14]:
lb = np.zeros(num_vars)
ub = np.full(num_vars, horizon)

# Binary variables have upper bound of 1
for i in range(num_binary_vars):
    ub[binary_idx(i)] = 1

# Integrality constraints (binary variables must be integer)
integrality = np.zeros(num_vars)
for i in range(num_binary_vars):
    integrality[binary_idx(i)] = 1

bounds = Bounds(lb=lb, ub=ub)

In [21]:
# Initialize constraint lists
A_ub = []
b_ub = []

# Precedence constraints: within each customer, appointments must be done in order
for customer_id, customer in enumerate(customers_data):
    for appt_id in range(len(customer) - 1):
        curr_idx = start_var_idx(customer_id, appt_id)
        next_idx = start_var_idx(customer_id, appt_id + 1)
        duration = customer[appt_id][1]

        constraint = np.zeros(num_vars)
        constraint[curr_idx] = 1
        constraint[next_idx] = -1

        A_ub.append(constraint)
        b_ub.append(-duration)

In [22]:
# Makespan constraints: makespan >= end time of last appointment for each customer
print("\nBuilding makespan constraints:")
for customer_id, customer in enumerate(customers_data):
    last_appt_id = len(customer) - 1
    last_appt_idx = start_var_idx(customer_id, last_appt_id)
    last_duration = customer[last_appt_id][1]

    constraint = np.zeros(num_vars)
    constraint[last_appt_idx] = 1
    constraint[makespan_idx()] = -1

    A_ub.append(constraint)
    b_ub.append(-last_duration)


Building makespan constraints:


In [23]:
# No-overlap constraints using big-M method
M = horizon  # Big-M value

for pair_idx, (i, j) in enumerate(agent_appt_pairs):
    binary_var_idx = binary_idx(pair_idx)

    cust_i, appt_i, agent_i, duration_i = appointment_info[i]
    cust_j, appt_j, agent_j, duration_j = appointment_info[j]

    constraint1 = np.zeros(num_vars)
    constraint1[i] = 1
    constraint1[j] = -1
    constraint1[binary_var_idx] = M

    A_ub.append(constraint1)
    b_ub.append(M - duration_i)

    constraint2 = np.zeros(num_vars)
    constraint2[j] = 1
    constraint2[i] = -1
    constraint2[binary_var_idx] = -M

    A_ub.append(constraint2)
    b_ub.append(-duration_j)

In [24]:
# Convert to numpy arrays and create a list of constraints
A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

constraints = [LinearConstraint(A_ub, -np.inf, b_ub)]

In [25]:
result = milp(c=c, integrality=integrality, bounds=bounds, constraints=constraints)

if result.success:
    print(f"Optimal schedule length: {result.x[makespan_idx()]:.1f} hours")

Optimal schedule length: 11.0 hours


In [26]:
def print_agent_schedule(solution, customers_data, appointment_info, num_agents):
    """Print the agent schedule in a readable format"""
    print("\n" + "=" * 60)
    print("CUSTOMER SERVICE SCHEDULE")
    print("=" * 60)

    makespan = solution[makespan_idx()]
    print(f"\nOptimal Schedule Length: {makespan:.1f} hours")

    # Extract start times
    print("\nCustomer Appointment Schedule:")
    for customer_id, customer in enumerate(customers_data):
        print(f"\nCustomer {customer_id}:")
        for appt_id, (agent, duration) in enumerate(customer):
            start = solution[start_var_idx(customer_id, appt_id)]
            end = start + duration
            print(
                f"  Appointment {appt_id}: Agent {agent}, Time [{start:.1f}, {end:.1f}]"
            )

    # Show agent schedules
    print("\n" + "-" * 60)
    print("AGENT SCHEDULES")
    print("-" * 60)

    for agent in range(num_agents):
        print(f"\nAgent {agent}:")

        # Find all appointments for this agent
        agent_appointments = []
        for idx, (customer_id, appt_id, appt_agent, duration) in enumerate(
            appointment_info
        ):
            if appt_agent == agent:
                start = solution[idx]
                agent_appointments.append(
                    {
                        "customer": customer_id,
                        "appointment": appt_id,
                        "start": start,
                        "end": start + duration,
                    }
                )

        # Sort by start time
        agent_appointments.sort(key=lambda x: x["start"])

        # Display timeline
        for appt in agent_appointments:
            print(
                f"  Customer {appt['customer']} Appointment {appt['appointment']}: [{appt['start']:.1f}, {appt['end']:.1f}]"
            )


if result.success:
    print_agent_schedule(result.x, customers_data, appointment_info, num_agents)
else:
    print("No feasible solution found!")
    print(f"Status: {result.status}")
    print(f"Message: {result.message}")


CUSTOMER SERVICE SCHEDULE

Optimal Schedule Length: 11.0 hours

Customer Appointment Schedule:

Customer 0:
  Appointment 0: Agent 0, Time [0.0, 3.0]
  Appointment 1: Agent 1, Time [4.0, 6.0]
  Appointment 2: Agent 2, Time [9.0, 11.0]

Customer 1:
  Appointment 0: Agent 0, Time [3.0, 5.0]
  Appointment 1: Agent 2, Time [5.0, 6.0]
  Appointment 2: Agent 1, Time [7.0, 11.0]

Customer 2:
  Appointment 0: Agent 1, Time [0.0, 4.0]
  Appointment 1: Agent 2, Time [6.0, 9.0]

------------------------------------------------------------
AGENT SCHEDULES
------------------------------------------------------------

Agent 0:
  Customer 0 Appointment 0: [0.0, 3.0]
  Customer 1 Appointment 0: [3.0, 5.0]

Agent 1:
  Customer 2 Appointment 0: [0.0, 4.0]
  Customer 0 Appointment 1: [4.0, 6.0]
  Customer 1 Appointment 2: [7.0, 11.0]

Agent 2:
  Customer 1 Appointment 1: [5.0, 6.0]
  Customer 2 Appointment 1: [6.0, 9.0]
  Customer 0 Appointment 2: [9.0, 11.0]
